In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyarrow==15.0.0
!pip install numpy==1.25.2

In [ ]:
!pip install datasets sentencepiece --quiet
# !pip install transformers --quiet
# !pip install accelerate --quiet
!pip install transformers[torch] --quiet
!pip install tqdm --quiet

In [ ]:
import transformers
import numpy as np
import json
import os
from torch.utils.data import Dataset, random_split
import torch
from transformers import BertTokenizerFast, AutoTokenizer, AutoModelForSeq2SeqLM,  MBartTokenizer, MBartForConditionalGeneration
import re
import chardet
import pandas as pd
from tqdm import tqdm

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [ ]:
pretrain_model_name = "/content/drive/MyDrive/ABSA_Datasets/Model/test_model_attention_mask/save_model"

tokenizer = MBartTokenizer.from_pretrained(pretrain_model_name)

model = MBartForConditionalGeneration.from_pretrained(pretrain_model_name)

model.resize_token_embeddings(len(tokenizer))

In [ ]:
def generate_text(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    input_ids = input_ids.to(device)
    model.to(device)

    outputs = model.generate(
        input_ids,
        num_beams=4,
        do_sample=True,
        temperature=1.2,
        top_k=50,
        top_p=0.95,
        num_return_sequences=4,
        max_length=50
    )

    #return tokenizer.decode(outputs[0])
    output_text = [tokenizer.decode(output, skip_special_tokens=False).replace("</s>", "").replace("<pad>","").strip() for output in outputs]
    return tuple(output_text)

# Example
#input_text = "จับตา OR ยอดใช้น้ำมันพุ่ง – Amazon มาแรง"
#print(generate_text(input_text))

In [ ]:
def generate_text(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    input_ids = input_ids.to(device)
    model.to(device)

    outputs = model.generate(
        input_ids,
        num_beams=4,
        do_sample=True,
        temperature=1.2,
        top_k=50,
        top_p=0.95,
        num_return_sequences=4,
        max_length=50,
        return_dict_in_generate=True,  # ให้คืนค่าเป็น dictionary
        output_scores=True  # เปิดให้คืนค่า logits ของ output แต่ละคำ
    )

    sequences = outputs.sequences  # Output sequences
    scores = outputs.scores  # Logits ของแต่ละ token

    # แปลง sequences เป็นข้อความ
    output_texts = [
        tokenizer.decode(seq).replace("</s>", "").replace("<pad>","").strip()
        for seq in sequences
    ]

    # คำนวณค่าความมั่นใจ (Softmax ที่ตำแหน่งสุดท้ายของ Sequence)
    confidences = []
    for seq_scores in scores:
        last_token_logits = seq_scores[-1]  # logits ของ token สุดท้าย
        probs = torch.nn.functional.softmax(last_token_logits, dim=-1)  # แปลง logits เป็น probability
        confidence = probs.max().item()  # เอาค่าความน่าจะเป็นสูงสุดเป็นค่าความมั่นใจ
        confidences.append(confidence)

    # คืนค่าเป็น tuple ของ (text, confidence)
    return list(zip(output_texts, confidences))

In [ ]:
output = generate_text("AMA ขนส่งขาขึ้น อัพรายได้ทะลุ 10%")

for text, confidence in output:
    print(f"{text}, {confidence:.3f}")

<SYMBOL> AMA <ASPECT> รายได้ <OPINION> ทะลุ <POS>, 1.000
<SYMBOL> AMA <ASPECT> ขนส่ง <OPINION> ขาขึ้น <POS>, 0.976
<SYMBOL> AMA <ASPECT> อัพรายได้ <OPINION> ทะลุ <POS>, 0.684
<SYMBOL> AMA <ASPECT> nan <OPINION> อัพรายได้ <POS>, 1.000


In [ ]:
# ทดสอบเรียกใช้งาน
responses = generate_text("AMA ขนส่งขาขึ้น อัพรายได้ทะลุ 10%")

# แสดงผลลัพธ์ทั้งหมด
for i, response in enumerate(responses, 1):
    print(f"output {i}: {response}")

output 1: ('<SYMBOL> AMA <ASPECT> รายได้ <OPINION> ทะลุ <POS>', 0.9995359182357788)
output 2: ('<SYMBOL> AMA <ASPECT> ขนส่ง <OPINION> ขาขึ้น <POS>', 0.9760790467262268)
output 3: ('<SYMBOL> AMA <ASPECT> อัพรายได้ <OPINION> ทะลุ <POS>', 0.6840067505836487)
output 4: ('<SYMBOL> AMA <ASPECT> nan <OPINION> อัพรายได้ <POS>', 0.9999276399612427)


In [ ]:
# Read a small part of the file to detect the encoding
with open("/content/drive/MyDrive/ABSA_Datasets/Text-to-Label-Dataset.csv", 'rb') as f:
    result = chardet.detect(f.read(10000))

# Print the detected encoding
print(result['encoding'])

# Read the CSV file using the detected encoding
df = pd.read_csv("/content/drive/MyDrive/ABSA_Datasets/Text-to-Label-Dataset.csv", encoding=result['encoding'])

utf-8


In [ ]:
df

,text,label
0,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,<SYMBOL> HMPRO<ASPECT> E-refune<OPINION> หนุน<...
1,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,<SYMBOL> HMPRO<ASPECT> หุ้น<OPINION> รับผลดี<POS>
2,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,<SYMBOL> HMPRO<ASPECT> Q1<OPINION> โต<POS>
3,ILM กรุงศรี พัฒนสิน มองอาจมี upside หลัง SSSG ...,<SYMBOL> ILM<ASPECT> SSSG<OPINION> เติบโตดี<POS>
4,IHLเป๋าตุงมกราทันที รับรู้รองเท้าแบรนด์ดัง,<SYMBOL> IHL<ASPECT> มกรา<OPINION> เป๋าตุง<POS>
...,...,...
794,หุ้น GLOBAL ราคาดีด หลังบอร์ดไฟเขียวจ่ายหุ้นปั...,<SYMBOL> GLOBAl<ASPECT> บอร์ด<OPINION> ไฟเขียว...
795,ILINKปักธงปี67พุ่ง10% ลุ้นผลประมูลงาน2พันล.,<SYMBOL> ILINK<ASPECT> nan<OPINION> พุ่ง<POS>
796,ILINKปักธงปี67พุ่ง10% ลุ้นผลประมูลงาน2พันล.,<SYMBOL> ILINK<ASPECT> ผลประมูลงาน<OPINION> ลุ...
797,TGE เผยบ.ย่อยชนะประมูลโรงไฟฟ้าขยะชุมชน ขนาด 9....,<SYMBOL> TGE<ASPECT> ประมูล<OPINION> ชนะ<POS>


In [ ]:
# Generate text and split into 4 separate columns
tqdm.pandas(desc='Generating texts...')
df[['generate-sample-1', 'generate-sample-2', 'generate-sample-3', 'generate-sample-4']] = pd.DataFrame(
    df['text'].progress_apply(lambda x: generate_text(x)).tolist(), index=df.index
)

# แสดงผลลัพธ์
print(df.head())

Generating texts...: 100%|██████████| 799/799 [03:00<00:00,  4.43it/s]

                                                text  \
0  HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...   
1  HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...   
2  HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...   
3  ILM กรุงศรี พัฒนสิน มองอาจมี upside หลัง SSSG ...   
4         IHLเป๋าตุงมกราทันที รับรู้รองเท้าแบรนด์ดัง   

                                               label  \
0  <SYMBOL> HMPRO<ASPECT> E-refune<OPINION> หนุน<...   
1  <SYMBOL> HMPRO<ASPECT> หุ้น<OPINION> รับผลดี<POS>   
2         <SYMBOL> HMPRO<ASPECT> Q1<OPINION> โต<POS>   
3   <SYMBOL> ILM<ASPECT> SSSG<OPINION> เติบโตดี<POS>   
4    <SYMBOL> IHL<ASPECT> มกรา<OPINION> เป๋าตุง<POS>   

                                   generate-sample-1  \
0  <SYMBOL> HMPRO <ASPECT> มาตรการ <OPINION> หนุน...   
1  <SYMBOL> HMPRO <ASPECT> มาตรการ <OPINION> หนุน...   
2  <SYMBOL> HMPRO <ASPECT> มาตรการ <OPINION> หนุน...   
3    <SYMBOL> ILM <ASPECT> เติบโต <OPINION> ดี <POS>   
4                     <s>en_XX <OPINION> ทันที

In [ ]:
# เปลี่ยนพาธของ data_folder
data_folder = '/content/drive/MyDrive/ABSA_Datasets/Model'

# สร้างพาธใหม่สำหรับไฟล์ผลลัพธ์
output_file_path = os.path.join(data_folder, 'mbart_evaluate.csv')

# บันทึกไฟล์ CSV
df.to_csv(output_file_path, index=False)

# แสดงผลลัพธ์ที่บันทึก
print(f'Results saved to {output_file_path}')

Results saved to /content/drive/MyDrive/ABSA_Datasets/Model/mbart_evaluate.csv
